# Environments

En JOS a los <b>procesos</b> se les llama <b>environments</b>. Asi que cuando hagamos referencia a un <b>environment</b> estaremos hablando de un <b>proceso</b> y viceversa.

## Estructura de un Environment 

La siguiente es la estructura de un proceso. Como podemos ver tiene diferentes cosas como el <b>env_id</b>, <b>env_parent_id</b> osea el <b>id</b> el proceso y el de su padre. Ademas tiene una estructura para guardarse los registros que utiliza, esto es <b>env_tf</b>. Luego tambien tiene una variable que guarda su estado, esta es <b>env_status</b>. Luego tenemos <b>env_runs</b> que guarda la cantidad de veces que este proceso fue corrido. Y por ultimo <b>env_pgdir</b> que es el directorio de paginas que usa este proceso.

```c
struct Env {
	struct Trapframe env_tf;	// Saved registers
	struct Env *env_link;		// Next free Env
	envid_t env_id;			// Unique environment identifier
	envid_t env_parent_id;		// env_id of this env's parent
	enum EnvType env_type;		// Indicates special system environments
	unsigned env_status;		// Status of the environment
	uint32_t env_runs;		// Number of times environment has run

	// Address space
	pde_t *env_pgdir;		// Kernel virtual address of page dir
};
```

#### Estados posibles para un Environment

```c
enum {
	ENV_FREE = 0,
	ENV_DYING,
	ENV_RUNNABLE,
	ENV_RUNNING,
	ENV_NOT_RUNNABLE
};
```

## Almacenamiento de los Environments

La forma que JOS trabaja almacenando los procesos es creando un array estatico de 1024 posiciones de tamanio del struct del environment.

<img src="Imagenes/environments.jpg">

Este array JOS lo llama <b>envs</b>:

```c
struct Env *envs = NULL;
```

Al proximo libre lo llama <b>env_free_list</b>:

```c
static struct Env *env_free_list;
```

Al que esta actualmente corriendo lo llama <b>curenv</b>. Viene de <b>current environment</b>:

```c
struct Env *curenv = NULL;
```

## Creacion de los Environments

Para crear este array hacemos los siguiente:

```c
void mem_init_envs() {
	// Creo el array que contendra todos los environments
	envs = boot_alloc(NENV * sizeof(struct Env));
	// Inicializo el array
	memset(envs, 0, NENV * sizeof(struct Env));
}
```

<hr>

# Elf Binary

Los binarios elf son un tipo particular de binarios que cuentan con la siguiente estructura.

<img src="Imagenes/binary_elf_estructura.svg" width="400" height="50">

```c
struct Elf {
	uint32_t e_magic;	// must equal ELF_MAGIC
	uint8_t e_elf[12];
	uint16_t e_type;
	uint16_t e_machine;
	uint32_t e_version;
	uint32_t e_entry;
	uint32_t e_phoff;
	uint32_t e_shoff;
	uint32_t e_flags;
	uint16_t e_ehsize;
	uint16_t e_phentsize;
	uint16_t e_phnum;
	uint16_t e_shentsize;
	uint16_t e_shnum;
	uint16_t e_shstrndx;
};
```

```c
struct Proghdr {
	uint32_t p_type;
	uint32_t p_offset;
	uint32_t p_va;
	uint32_t p_pa;
	uint32_t p_filesz;
	uint32_t p_memsz;
	uint32_t p_flags;
	uint32_t p_align;
};
```